In [ ]:
import napari
import os
import skimage
from skimage import io, measure
from skimage.io import imread
import numpy as np
import copy
import pandas as pd
import tifffile
from PIL import Image, ImageFont, ImageDraw 
import seaborn as sns
import matplotlib.pyplot as plt

import palettable.cartocolors.qualitative as qual_palettes

In [ ]:
#paths
codedir = '/Users/engje/Documents/Data/cycIF_TMAs'
regdir = f'{codedir}/Cropped'
#regdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/Cropped'

#segdir ='/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Analysis/cmIF_2021-05-03_PDAC/U54-TMA-9/Jenny/Segmentation/Mesmer_Segmentation'

os.chdir(codedir)
os.chdir('..')
from mplex_image import visualize as viz
from mplex_image import mpimage, analyze
os.chdir(codedir)
title_font = ImageFont.truetype('SansSerifBldFLF.otf',50)

In [ ]:
# RNA seq - done
# # Python to translate ensembl to gene name: done
# a = 'Tempus300'#'SingleEnd'#'PairedEnd'
# df_annot = pd.read_csv(f"results_annotation_{a}.csv",index_col=0)
# df_annot.head()

# df_tpm = pd.read_csv(f"results_TPM_{a}.csv",index_col=0)
# # if df_tpm.index[0].find('.')>-1: #periods
# #     df_tpm.index = [item.split('.')[0] for item in df_tpm.index]
# df_tpm.index = df_tpm.index.map(dict(zip(df_annot.gene_id,df_annot.gene_name)))
# print(len(df_tpm))
# print(len(df_tpm[~df_tpm.index.duplicated(keep='first')]))
# df_tpm_sum = df_tpm.groupby(by=df_tpm.index, axis=0).sum()
# print(len(df_tpm_sum))
# df_tpm_sum.to_csv(f"results_TPM_{a}_sum.csv")

# load data

In [ ]:
#load single cell data
os.chdir(codedir)
s_data = '20240520_NP-Tissues_FilteredMeanIntensity.csv'#"20230326_U54-TMA_CombinedCelltypes_PDAC.csv"#'20220721_U54-TMA_CombinedCelltypes_PDAC.csv'


df_mi = pd.read_csv(f'{s_data}',index_col=0)
df_mi['cellid'] = [item.split('cell')[1] for item in df_mi.index]

df_mi.head()

In [ ]:
# make the intensity data (done)
# ls_file = ['features_mTMA2-4_Ecad_patched_MeanIntensity_Shape_DAPI1_DAPI6_registeredimages.csv',
#  'features_mTMA2-5_Ecad_patched_MeanIntensity_Shape_DAPI1_DAPI6_registeredimages.csv']
# df_all = pd.DataFrame()
# for s_file in ls_file:
#     df = pd.read_csv(s_file,index_col=0)
#     df_all = pd.concat([df_all,df])
# df_all = df_all.loc[:,df_all.dtypes=='float64']
# df_all = df_all.loc[:,~df_all.columns.str.contains('DAPI')]
# df_all = df_all.loc[:,~df_all.columns.str.contains('cellmem2p25')]
# df_all.columns = [item.split('_')[0] for item in df_all.columns]
# df_all = df_all.loc[:,~df_all.columns.duplicated(keep=False)]
# df_all.drop('nuclei',axis=1,inplace=True)
# df_mi.loc[:,['seurat_clusters']].merge(df_all,left_index=True,right_index=True).to_csv('tma_int.data.csv')

#load intensity data
# df_clust = df_mi.loc[:,(df_mi.dtypes=='float64') | (df_mi.columns=='HDBSCAN')].drop(['DAPI_X','DAPI_Y','0','1'],axis=1) #pd.read_csv('tma_int.data.csv',index_col=0)

In [ ]:
# d_replace = dict(zip(df_mi.HDBSCAN,df_mi.leidencelltype5))
# d_color = dict(zip(['endothelial', 'epithelial', 'fibroblast', 'immune', 'stromal'],sns.color_palette()[0:5]))

In [ ]:
# %matplotlib inline 
# ls_drop =  [] #['LamAC','RAD51', 'gH2AX','pMYC','pRPA','CD4', 'CD8','ColI']
# df_plot = df_clust.drop(ls_drop,axis=1).groupby('HDBSCAN').mean()
# row_colors = df_plot.index.map(d_replace).map(d_color)
# g = sns.clustermap(df_plot,z_score=1,vmin=-2,vmax=2,cmap='viridis',figsize=(6,5),xticklabels=1,yticklabels=1,
#                    dendrogram_ratio=0.08, cbar_pos=(-0.01, 0.92, 0.03, 0.10),row_colors=row_colors,
#                   cbar_kws={'label':'Z-score'})
# for label in d_color.keys():
#     g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
#                             label=label, linewidth=0)
# g.ax_row_dendrogram.legend(loc="right",title='Lineage',ncol=1,bbox_to_anchor=(0.1,0.45))
# g.ax_heatmap.set_ylabel('Annotated Cell Type')
# g.ax_heatmap.set_xlabel('Marker',labelpad=-10)
# g.savefig('clusters_heatmap.png',dpi=300)

In [ ]:
# ## QC/ identify high expressing cores

# # df_clust['slide_scene'] = [item.split('_cell')[0] for item in df_clust.index]
# # df_test = df_clust.loc[:,['slide_scene','HDBSCAN']].groupby('slide_scene').value_counts().unstack() #normalize=True

# # df_core[df_core.Old_Pt_ID== '4832-T'] #CD68+ with Ecad+ or Vim+ cells, cell-in-cell
# # df_core[df_core.Old_Pt_ID== '4741-T'] 
# # df_core[df_core.Old_Pt_ID== '4806-T'] 
# # df_core[df_core.Old_Pt_ID== '4832-T'] 
# # df_core[df_core.Old_Pt_ID== '4612-T'] 
# # df_core.sort_values(by='PurIST_Score',ascending=False)[0:10]
# # df_test.sort_values('CK17+ tumor',ascending=False)
# # df_test.sort_values('CK19 low CD44+ tumor',ascending=False)
# test_cluster = 'CD68+ immune'#'CK5+ PDPN+ tumor'#'aSMA+ blood vessel'
# df_clust['slide_scene'] = [item.split('_cell')[0] for item in df_clust.index]
# df_test = df_clust.loc[:,['slide_scene','HDBSCAN']].groupby('slide_scene').value_counts().unstack() #normalize=True
# df_test.sort_values(by=test_cluster,ascending=False)[0:4]

In [ ]:
# identify patients
# d_pt = dict(zip(df_core.coor_mplexable,df_core.Old_Pt_ID))
# df_test['PT'] = df_test.index.map(d_pt)
# # s_col = 'CD68+ immune'
# # df_test.loc[:,['PT',s_col]].sort_values(by=s_col,ascending=False)

## Load a core

scatterplot clustering results

In [ ]:
# #colors
# %matplotlib inline
# color_palette =  qual_palettes.Bold_8.mpl_colors +  qual_palettes.Safe_8.mpl_colors + qual_palettes.Pastel_10.mpl_colors
# d_color = dict(zip(df_mi.HDBSCAN2.unique(),color_palette))
# #d_color.update({'intraductal cells':'lightgray', 'small round cells':'lightgray'})
# #ls_scene = ['U54-TMA-9_sceneH03', 'U54-TMA-9_sceneF03', 'U54-TMA-9_sceneB11'] #tumor K
# #ls_scene = ['U54-TMA-9_sceneB07', 'U54-TMA-9_sceneH11', 'U54-TMA-9_sceneG07'] # b cell K
# #ls_scene = ['U54-TMA-9_sceneE09','U54-TMA-9_sceneC04','U54-TMA-9_sceneB07',] # pericyte
# #ls_scene = ['U54-TMA-9_sceneH11', 'U54-TMA-9_sceneE02', 'U54-TMA-9_sceneI06'] #endothelial
# #ls_scene = ['U54-TMA-9_sceneI10','U54-TMA-9_sceneG05','U54-TMA-9_sceneC06'] # B cell change to FB
# #ls_scene = ['U54-TMA-9_sceneE10','U54-TMA-9_sceneG05','U54-TMA-9_sceneB07','U54-TMA-9_sceneH11','U54-TMA-9_sceneC11','U54-TMA-9_sceneG09','U54-TMA-9_sceneC06',] #examples: aSMA+ FB, PDPN+ tumor
# ls_scene = df_test.sort_values(by=test_cluster,ascending=False)[0:4].index
# #scatterplot the celltypes
# for s_slide in ls_scene:
#     s_slide = 'U54-TMA-9_sceneE04'
#     df_scene = df_mi[df_mi.slide==s_slide]
#     fig,ax=plt.subplots(figsize=(10,9),dpi=200)
#     sns.scatterplot(data=df_scene, x='DAPI_X', y='DAPI_Y', hue='HDBSCAN2',palette=d_color,ax=ax,
#                    s=8,edgecolor=None)
#     ax.legend(bbox_to_anchor=(1.01,0.9))
#     ax.set_title(f"{s_slide}", fontsize=16) 
#     ax.axis('equal')
#     ax.set_ylim(ax.get_ylim()[::-1])
#     ax.set_xticklabels('')
#     ax.set_yticklabels('')
#     break
    

In [ ]:
os.listdir(regdir)

# napari viewer

In [ ]:
# crops/local
from skimage import transform, io, segmentation
ls_file = [#'ReRegistered-R11_S100.AR.LaminB2.CD45_NP9-IDC2-Scene-001_c1_ORG.tif'
 'ReRegistered-R11_S100.AR.LaminB2.CD45_NP9-IDC2-Scene-003_c1_ORG.tif',
 'ReRegistered-R1_PCNA.CD8.PD1.CK19_NP9-IDC2-Scene-003_c5_ORG.tif']
s_marker = 'DAPI11'
#s_file = 'ReRegistered-R0_blank.blank.blank_NP9-IDC2-Scene-001_c2_ORG.tif'
#s_marker = 'R0c2'

#load images
#print(s_slide)
os.chdir(f'{regdir}')

viewer = napari.Viewer()
for s_file in ls_file:
    img = io.imread(s_file)
    viewer.add_image(img,name=s_marker,rgb=False,blending='additive',colormap='blue',contrast_limits = (np.quantile(img,0),(np.quantile(img,0.9999)+1)*1.5))

#scale bar
viewer.scale_bar.visible = True
title_text = s_file

#viz.add_slide_name(viewer, title_text,title_font, s_layer='DAPI11')



In [ ]:
print('done')

In [ ]:
# # # add all the clusters
# df_scene = df_mi[df_mi.slide==s_slide] #[:-1]
# s_col = 'HDBSCAN2' # 'leidencelltype5' #
# for idx,i_clust in enumerate(sorted(df_scene.loc[:,s_col].unique())): 
#     viz.add_cluster(viewer,df_scene,s_col,i_clust,idx,label_corrected)

## ROI selection

select areas of the tissue to analyze or exclude

In [ ]:
s_slide = s_file.split('-Scene-')[0].split('_')[-1]
s_scene = s_file.split('-Scene-')[1].split('_')[0]
s_slide_scene = f'{s_slide}_scene{s_scene}'
df_scene = df_mi[df_mi.slide_scene==s_slide_scene]
print(s_slide_scene)

In [ ]:
# add points at centroids
s_crop = 'x0y0'
points,x_arr = viz.add_points(s_crop,df_scene,s_slide)
viewer.add_points(points)

In [ ]:
#draw shape, then run this cell (after every new shape)

verts = viewer.layers['Shapes'].data[0]
b_poly = measure.points_in_poly(points, verts)
point_properties = {
    'label': np.array(df_scene.loc[df_scene.index.str.contains(s_slide),'cellid']),
    'in_poly' : np.array(b_poly)
}
points_layer = viewer.add_points(points, properties=point_properties, face_color='in_poly',face_color_cycle=['magenta', 'black'],edge_width=0)


In [ ]:
# specify if points to exclude are ...
# in polygon (might be black or magenta)
df_exclude = x_arr.loc[b_poly]
# out of polygon (again, color can change)
#df_exclude = x_arr.loc[~b_poly]

In [ ]:
# # add another ROI
# # more in poly
# df_exclude = pd.concat([df_exclude,x_arr.loc[b_poly]])
# # more out of poly
# #df_exclude = pd.concat([df_exclude,x_arr.loc[~b_poly]])

In [ ]:
# save
df_exclude.to_csv(f'exclude_{s_slide}_Normal4.csv')

In [ ]:
df_exclude

# not used

## Equalized display range

View multiple tissues with same markers and display range

In [ ]:
for s_tissue in ['Liver_Met','Lung_Met','Normal_Lung','Normal_Liver']:
    ls_slide = df_core.loc[df_core.Tissue==s_tissue,['coor_mplexable','Old_Pt_ID','pORG_Score','Cohort']].coor_mplexable.unique()
    break

In [ ]:
df_core[df_core.Old_Pt_ID=='4702-T'] #sceneI05: is this one labelled wrong?. it does have CD31 and CD68

In [ ]:
ls_slide = ['U54-TMA-9_sceneG05', 'U54-TMA-9_sceneE10', 'U54-TMA-9_sceneH12',
       'U54-TMA-9_sceneE03', 'U54-TMA-9_sceneB06', 'U54-TMA-9_sceneE02',
       'U54-TMA-9_sceneH01', 'U54-TMA-9_sceneH09', 'U54-TMA-9_sceneI05',
       'U54-TMA-9_sceneB11', 'U54-TMA-9_sceneB12', 'U54-TMA-9_sceneD02',
       'U54-TMA-9_sceneD05', 'U54-TMA-9_sceneE04', 'U54-TMA-9_sceneG07'] # ck5
ls_slide = ['U54-TMA-9_sceneA10', 'U54-TMA-9_sceneB11', 'U54-TMA-9_sceneE03',
       'U54-TMA-9_sceneG10', 'U54-TMA-9_sceneC11', 'U54-TMA-9_sceneE11',
       'U54-TMA-9_sceneB02', 'U54-TMA-9_sceneE10', 'U54-TMA-9_sceneA09',
       'U54-TMA-9_sceneC09', 'U54-TMA-9_sceneB09', 'U54-TMA-9_sceneB06',
       'U54-TMA-9_sceneC08', 'U54-TMA-9_sceneG09', 'U54-TMA-9_sceneD08',
       'U54-TMA-9_sceneC06', 'U54-TMA-9_sceneI10', 'U54-TMA-9_sceneA07',
       'U54-TMA-9_sceneF03', 'U54-TMA-9_sceneF02', 'U54-TMA-9_sceneD02',
       'U54-TMA-9_sceneB12', 'U54-TMA-9_sceneE04', 'U54-TMA-9_sceneG01',
       'U54-TMA-9_sceneF07', 'U54-TMA-9_sceneD11', 'U54-TMA-9_sceneD09',
       'U54-TMA-9_sceneE09', 'U54-TMA-9_sceneE05', 'U54-TMA-9_sceneE02',
       'U54-TMA-9_sceneF04', 'U54-TMA-9_sceneF11', 'U54-TMA-9_sceneC10',
       'U54-TMA-9_sceneG02', 'U54-TMA-9_sceneC04', 'U54-TMA-9_sceneB04',
       'U54-TMA-9_sceneH01', 'U54-TMA-9_sceneI06', 'U54-TMA-9_sceneA05'] #CK17

In [ ]:
# equalized display range for making figures
#ls_slide = ['mTMA2-4_sceneA04','mTMA2-4_sceneA01',]
d_marker = {'DAPI1':0,'Ecad':50,'panCK':150,'CK19':150,'Vim':35,#'CD45':(2,30),'CD31':(3,150),
           'CK17':(1,10),'CK5':80,'Ki67':80} #,'Ki67':80,'"panCK':150,
d_marker = {'DAPI1':0,'Ecad':50,#'panCK':150,'CK19':150,'Vim':35,#'CD45':(2,30),'CD31':(3,150),
           'CK17':(1,10),'CK5':80}#,'Ki67':80} 
os.chdir(regdir)
for s_slide in ls_slide: 
    #s_slide = 'U54-TMA-9_sceneI05'#'U54-TMA-9_sceneG09'
    print(s_slide)
    for s_file in os.listdir():
        if s_file.find(s_slide) > -1:
            if s_file.find('Ecad') == -1:
                s_crop = s_file.split('_')[2]
    viewer = napari.Viewer()
    label_image = viz.load_marker(viewer,s_crop,s_slide,d_marker)
    #scale bar
    viewer.scale_bar.visible = True
    se_core = df_core[df_core.coor_mplexable==s_slide].iloc[0]
    try: 
        title_text = se_core.coor_mplexable + ' ' + se_core.Old_Pt_ID + ' pORG=' + str(se_core.pORG_Score) + ' ' + se_core.Cohort#f'{s_slide}'
    except:
        title_text = se_core.coor_mplexable + ' ' + se_core.Old_Pt_ID #+ ' ' + se_core.Tissue
    viz.add_slide_name(viewer, title_text,title_font, s_layer='DAPI1')
    #break

In [ ]:
print('done')

## Viewing markers at full resolution (server)

make sure to mount server (cmd-k)

requires access

In [ ]:
#functions
def add_slide_name(viewer, title_text, s_layer='DAPI1'):
    name_img  = np.zeros(viewer.layers[s_layer].data.shape)
    my_image=Image.fromarray(name_img)
    image_editable = ImageDraw.Draw(my_image)
    image_editable.text(xy=(150,150), text=title_text, fill=(200), font=title_font)
    viewer.add_image(np.asarray(my_image), blending='additive',visible=True,name=title_text)


def make_title_text(df_core,s_slide):
    s_patient = df_core.loc[df_core.coor_mplexable==s_slide,"SpecimenID"].iloc[0]
    s_porg = str(df_core.loc[df_core.coor_mplexable==s_slide,"pORG_Score"].iloc[0].round(3))
    if df_core.loc[df_core.coor_mplexable==s_slide].Liver_Met_Radiographic.iloc[0] == 'metastasis':
        s_met = 'liver'
    elif df_core.loc[df_core.coor_mplexable==s_slide].Lung_Met_Raidographic.iloc[0] == 'metastasis':
        s_met = 'lung'
    else:
        s_met = ''
    title_text = f'{s_slide}  {s_patient} {s_met} pORG={s_porg}'
    return(title_text)

In [ ]:
## viewing markers at full resolution (server)
regdir_server = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2021-05-03_PDAC/SubtractedRegisteredImages'
segdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2021-05-03_PDAC/Segmentation/U54-TMA-9_CellposeSegmentation'

from skimage.io import imread
ls_rpa = ['U54-TMA-9_sceneG05', 'U54-TMA-9_sceneB06',
          'U54-TMA-9_sceneC12',#'mTMA2-4_sceneA01'
 ]

ls_marker = ['pMYC','DAPI1','CK19','Ecad']
ls_marker = ['pMYC','DAPI1','PDPN','Ecad','panCK','Vim','CK19','CK5','CK17']
for s_slide in ls_rpa:
    regdir_slide = f'{regdir_server}/{s_slide}'
    os.chdir(regdir_slide)
    df_img = mpimage.parse_org()
    #update dapis
    d_out = {item:'DAPI' + df_img.loc[item,'round_num'].astype('int').astype('str') for item in df_img[df_img.marker=='DAPI'].index}
    for s_index, s_dapi in d_out.items():
        df_img.loc[s_index,'marker'] = s_dapi
    filenames = df_img[df_img.marker.isin(ls_marker)].index
    viewer = napari.Viewer()
    for i, filename in enumerate(filenames):
        if filename.find('Registered') > -1:
            s_index = f"Registered{filename.split('Registered')[1]}"
            s_marker = df_img.loc[s_index,'marker']
            img = imread(s_index)
            q99 = np.quantile(img,0.998)
            if s_marker.find('DAPI') > -1:
                viewer.add_image(img, contrast_limits=[1000,65000], multiscale=False, blending='additive',visible=True,name=s_marker,colormap='blue')
            elif s_marker == 'CK19' or s_marker == 'Ecad':
                viewer.add_image(img, contrast_limits=[750,q99*1.5], multiscale=False, blending='additive',visible=False,name=s_marker)
            else:
                viewer.add_image(img, contrast_limits=[0,q99*1.5], multiscale=False, blending='additive',visible=False,name=s_marker)
    os.chdir(segdir)
    labels = io.imread(f'{s_slide}_Ecad_nuc30_cell30_matched_exp5_CellSegmentationBasins.tif')
    viewer.add_labels(labels,blending='additive')
#     nuclabels = io.imread(f'{s_slide}_nuclei0.5_NucleiSegmentationBasins.tif')
#     viewer.add_labels(nuclabels,blending='additive')
    add_slide_name(viewer, s_slide,s_layer='DAPI1')
    viewer.scale_bar.visible = True
    break


###  move images around on one drive

copy cores analyzed by zinab

In [ ]:
## server (works way better)
# import shutil
# cropdir = '/Volumes/BCC_Chin_Lab_RDS/ChinData/Cyclic_Workflow/cmIF_2022-03-25_mTMA2/Cropped'
# os.chdir(cropdir)
# ls_scene = df_mi.slidescene.unique()[2::]
# df_img = pd.DataFrame(index=os.listdir())
# df_img['scene'] = [item.split('_')[0] + '_' + item.split('_')[1] for item in df_img.index]
# for s_scene in ls_scene:
#     print(s_scene)
#     ls_img = df_img[df_img.scene==s_scene].index
#     for s_img in ls_img:
#         source = s_img
#         destination = f'{regdir}/{s_img}'
#         shutil.copy(source, destination)
#     #break
# os.chdir(codedir)

In [ ]:
## one drive (works poorly)
# ls_scene = df_mi.slidescene.unique()
# os.chdir('/Users/engje//Oregon Health & Science University - Zinab Doha - cyclic_IF_ZD_JE/Cropped1')
# import shutil
# df_img = pd.DataFrame(index=os.listdir())
# df_img['scene'] = [item.split('_')[0] + '_' + item.split('_')[1] for item in df_img.index]
# for s_scene in ls_scene:
#     print(s_scene)
#     ls_img = df_img[df_img.scene==s_scene].index
#     for s_img in ls_img:
#         source = s_img
#         destination = f'../Cropped/{s_img}'
#         shutil.copy(source, destination)
#os.chdir(codedir)